In [1]:
import os
import sys
import numpy as np
import re
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from os import listdir
import pandas as pd
from keras.layers import Dense, Dropout, Input, add, Conv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose,Activation, Concatenate
from tensorflow import keras
from time import time
from keras import backend as K
from PIL import Image, ImageDraw, ImageOps
import segmentation_models as sm
from sklearn.cluster import KMeans

# import keras_retinanet
from lacmus.keras_retinanet import models
from lacmus.keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from lacmus.keras_retinanet.utils.visualization import draw_box, draw_caption
from lacmus.keras_retinanet.utils.colors import label_color

import networkx as nx
from itertools import product
from shortestpaths import k_shortest_paths



Segmentation Models: using `keras` framework.


Constatns

In [2]:
IMG_SIZE = (512, 512) #(1024,1024)

Loss

In [3]:
def dice_coef(y_true, y_pred):

    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0) 
 
def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def combined_loss(y_true, y_pred):
    dice_loss = dice_coef_loss(y_true, y_pred)
    bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return 0.5 * bce_loss + 0.5 * dice_loss

In [4]:
def placeMaskOnImg(img, mask, color):
    color = [i/255.0 for i in color]
    np.place(img[:, :, :], mask[:, :, :] >= 0.5, color)
    return img

def make_pred_good(pred):
#     pred = pred.numpy()
    pred = pred[0][:, :, :]
    pred = np.repeat(pred, 3, 2)
    return pred

Road model load 

In [ ]:
BACKBONE = 'resnet50'
n_classes = 1
activation = 'sigmoid'
preprocess_input = sm.get_preprocessing(BACKBONE)
road_model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

In [ ]:
road_weights_path =  'road_model__120epochs.h5'
road_model.load_weights(road_weights_path)

Human model load

In [9]:
human_model_path = 'human_model_120epochs.h5'
human_model = models.load_model(human_model_path, backbone_name='resnet50')

Coords to pixels

In [10]:
def get_pixels_by_coords(t_lat,t_lon, rb_lat, rb_lon):
    b_lat, b_lon = 56.4, 37.93937397222222
    lat, lon = 56.3935451, 37.9920535
    h, w = 86//2, 386//2
    lat_coeff = h/(b_lat-lat)
    lon_coeff = w/(b_lon-lon)

    pix_h, pix_w = int((rb_lat-t_lat)*lat_coeff), int((rb_lon-t_lon)*lon_coeff)

    return pix_h, pix_w

Creating graph out of the mask and finding three shortest parts to nearest localties

In [11]:
from itertools import product
def create_graph(matrix, threshold=10):
    rows, cols = matrix.shape
    graph = nx.DiGraph()

    for i, j in product(range(rows), range(cols)):
        if matrix[i, j] == 1:
            graph.add_node((i, j))

            for x, y in product(range(i - threshold, i + threshold + 1), range(j - threshold, j + threshold + 1)):
                if 0 <= x < rows and 0 <= y < cols and matrix[x, y] == 1 and ((x - i) ** 2 + (y - j) ** 2) <= threshold ** 2:
                    graph.add_edge((i, j), (x, y), weight=((x - i) ** 2 + (y - j) ** 2) ** 0.5)

    return graph

def find_three_shortest_paths(matrix, start, end, threshold=10):
    graph = create_graph(matrix, threshold)
    paths = list(k_shortest_paths(graph, start, end, 1, 'd'))#, weight='weight'
    return paths#result_matrix

Processing coordinates from UAV aerial photo and getting sattelite map from this coordinates, also decorating this map with labels and markdowns

In [12]:
import requests
from matplotlib import pyplot as plt
import cv2
import numpy as np
from exif import Image
import json
import folium
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
from geopy.exc import GeocoderTimedOut
from selenium import webdriver
import os
import time

def decimal_coords(coords, ref): # Convert GPS to degrees
    decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
    if ref == "S" or ref == "W":
        decimal_degrees = -decimal_degrees
    return decimal_degrees

def image_coordinates(image_path): # Get GPS
    with open(image_path, "rb") as src:
        img = Image(src)
        if img.has_exif:
            try:
                lat = decimal_coords(img.gps_latitude, img.gps_latitude_ref)
                lon = decimal_coords(img.gps_longitude, img.gps_longitude_ref)
                return (lat, lon)
            except AttributeError:
                print("No coordinates")
        else:
            print("The image has no EXIF information")

#getting map by coords and marking recognized humans on the map
def place_human_labels(amount):
    driver = webdriver.Firefox() 
    driver.get("file://" + os.path.abspath("map.html"))
    time.sleep(1)
    map_path = "map.png"
    driver.save_screenshot(map_path)
    driver.quit()
    mask_m = plt.imread(map_path)
    plt.imsave(fname=map_path[:-3]+'jpg',arr = mask_m)
    del mask_m
    # remove the png file, but keep the 8-bit mask
    os.remove(map_path)
    satellite_map = plt.imread("map.jpg")
    base_h, base_w = satellite_map.shape[0]//2, satellite_map.shape[1]//2
    img = cv2.rectangle(satellite_map, (base_w-20, base_h-20, 40, 40),(255,0,0),2)

    img = cv2.putText(satellite_map, str(amount), (base_w-10, base_h+10), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 2)
    return satellite_map

#getting nearest localties and marking them
def get_map(image_source, amount):
    api_key = "hidden"
    latitude, longitude = 56.07461313,34.43509312 #basic coords for my screen resolution
    location = f'{latitude}, {longitude}'
    radius = 6000
    zoom = 14
    type = "city|village"
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type={type}&key={api_key}"

    response = requests.get(url)
    results = response.json()["results"]

    print(f"Found {len(results)} nearby cities and villages.")

    places = []

    for result in results:
    
        place_id = result["place_id"]
        
        fields = "name,formatted_address,geometry"
        url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields={fields}&key={api_key}"

        response = requests.get(url)
        result = response.json()["result"]

    
        place = {
            "name": result["name"],
            "address": result["formatted_address"],
            "lat": result["geometry"]["location"]["lat"],
            "lng": result["geometry"]["location"]["lng"]
        }

        places.append(place)
        
    geolocator = Nominatim(user_agent="myGeocoder")


    m = folium.Map(location=[latitude, longitude], zoom_start=13, tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}", attr="Esri")
    m_unlabelled = folium.Map(location=[latitude, longitude], zoom_start=13, tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}", attr="Esri")
    folium.Marker([latitude, longitude], popup="Your Location", icon=folium.Icon(color="blue")).add_to(m)
    
    
    for place in places:
        
        print(place)
        nearest_lat, nearest_lon = place["lat"], place["lng"]
        folium.Marker([nearest_lat, nearest_lon], popup="Nearest Settlement", icon=folium.Icon(color="red")).add_to(m)

    m.save("map.html")
    m_unlabelled.save("map_unlabelled.html")
    driver = webdriver.Firefox() 
    driver.get("file://" + os.path.abspath("map_unlabelled.html"))
    time.sleep(1)
    map_path = "map_unlabelled.png"
    driver.save_screenshot(map_path)
    driver.quit()
    mask_m = plt.imread(map_path)
    plt.imsave(fname=map_path[:-3]+'jpg',arr = mask_m)
    del mask_m
    # remove the png file, but keep the 8-bit mask
    os.remove(map_path)
    labelled_map = place_human_labels(amount)
    satellite_map = plt.imread("map_unlabelled.jpg")
    return labelled_map, satellite_map


Finding, which part of recognized roads is closest to the recognized humans

In [13]:
def find_closest(matrix, start_coord):
    mask = np.ma.masked_not_equal(matrix, 1)
    indices = np.argwhere(mask.mask == False)
    distances = [np.sqrt((i[0]-start_coord[0])**2 + (i[1]-start_coord[1])**2) for i in indices]
    closest_index = distances.index(min(distances))
    return tuple(indices[closest_index])

Making input picture smoother for better recognition

In [14]:
def interpolate_image(image):
    pixels = np.array(image)
    reshaped_pixels = pixels.reshape((-1, 3))
    n_clusters = 8
    kmeans = KMeans(n_clusters=n_clusters, n_init=1).fit(reshaped_pixels)
    new_pixels = kmeans.cluster_centers_[kmeans.labels_].astype(np.uint8)
    new_image =  new_pixels.reshape(pixels.shape)  #Image.fromarray(new_pixels.reshape(pixels.shape))
    return new_image

In [15]:
import gradio as gr

c:\Users\Mitty\anaconda3\envs\ml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Getting map with right human labels

In [16]:
def get_satellite_map_from_image(image_source, amount):
        amount = int(amount['label'].split()[0])
        img = get_map(image_source, amount)
        return img

Creating borders for better patching

In [17]:
def create_borders_UI(img, desired_size=IMG_SIZE):
    delta_w = (img.shape[0] if img.shape[0]%desired_size[0]==0 else ((img.shape[0]//desired_size[0])+1)*desired_size[0]) - img.shape[0]
    delta_h = (img.shape[1] if img.shape[1]%desired_size[1]==0 else ((img.shape[1]//desired_size[1])+1)*desired_size[1]) - img.shape[1]
    img = cv2.copyMakeBorder(img, 0, delta_w, 0, delta_h, cv2.BORDER_CONSTANT, value=1)
    return img 

Processing staellite map image to recognize roads

In [18]:
def process_road_image(image_source, patch_size=(800,800)):
    img = interpolate_image(image_source)
    img = (image_source/255.0).astype(np.float32)
    img = np.expand_dims(cv2.resize(img, (512, 512)), 0)
    
    predicted_images = []
    img_patches = []
    
    for y in range(0, img.shape[0], patch_size[1]):
        for x in range(0, img.shape[1], patch_size[0]):
            img_patch = img[y:y + patch_size[1], x:x + patch_size[0]]
            img_patches.append(cv2.resize(img_patch, (512, 512)))     
    for i in range(len(img_patches)):
        patch = np.expand_dims(img_patches[i], axis = 0)
        pred = road_model.predict(patch)
        pred = pred[0][:, :, :]
        pred_o = pred
        pred_o = np.array(pred)
        pred_o = pred_o[:, :, 0].reshape((512, 512))
        
        t_mask = pred_o
        mean_mask = np.mean(t_mask)
        np.place(pred_o[:, :], t_mask[:, :] < mean_mask*3, 0)
        t_mask = pred_o
        np.place(pred_o[:, :], t_mask[:, :] >= mean_mask*3, 1)
        
        predicted_images.append(pred_o)

    vis_h_pred = []
    vis_h_img = []
    h_steps = int(img.shape[1]/patch_size[1])
    v_steps = int(img.shape[0]/patch_size[1])
    h_1 = 0
    h_2 = h_steps
    for _ in range(v_steps):
        vis_h_pred.append(cv2.hconcat([predicted_images[i] for i in range(h_1, h_2)]))
        vis_h_img.append(cv2.hconcat([img_patches[i] for i in range(h_1, h_2)]))
        h_1 = h_2
        h_2 += h_steps
    predicted_image = cv2.vconcat([vis_h_pred[i] for i in range(len(vis_h_pred))])
    img = cv2.vconcat([vis_h_img[i] for i in range(len(vis_h_img))])
    

    mask_on_img = placeMaskOnImg(img[0], np.repeat(np.expand_dims(predicted_image,-1), 3, 2),(66, 255, 73))
    return 'Predicted Mask', predicted_image, 'Predicted Mask on Image', mask_on_img

Processing UAV image to recognize humans

In [19]:
def process_drone_image(image_source, patch_size=IMG_SIZE):
    labels_to_names = {0: 'Human'}
    
    image = image_source
    
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    image = preprocess_image(image)
    image, scale = resize_image(image)

    boxes, scores, labels = human_model.predict_on_batch(np.expand_dims(image, axis=0))
    
    boxes /= scale
    humans = 0
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < 0.5:
            break
        color = label_color(label)
        humans+=1
        
        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
        
    draw_conv = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    
    img = (image_source/255.0)
    img = np.expand_dims(cv2.resize(img, (512, 512)), 0)
    img = img.astype(np.float32)
    pred = road_model.predict(img)
    pred = pred[0][:, :, :]
    
    mask = pred
    t_mask = mask
    np.place(mask[:, :, :], t_mask[:, :, :] < 0.0005, 0)
    t_mask = mask
    np.place(mask[:, :, :], t_mask[:, :, :] >=0.0005, 1)
    pred = mask
    
    pred = np.repeat(pred, 3, 2) 
    mask_on_img = placeMaskOnImg(img[0], pred,(66, 255, 73))
    return 'Predicted humans', draw_conv , f'{humans} humans were found on image', humans  #mask_on_img

Processing staellite map image with recognized roads to find out, which is the shortest ways for recognized humans to get to the nearest localties by recognized roads

In [108]:
def process_road_way_image(image_source, mask, coords, patch_size=IMG_SIZE):
    img = (image_source/255.0).astype(np.float32)   
    img = img[:, :, 0].reshape((512, 1024))
    t_mask = img
    np.place(img , t_mask < 50, 0)
    t_mask = img
    np.place(img , t_mask >= 50, 1)
    base_h, base_w = img.shape[0]//2, img.shape[1]//2
    matrix = img
    start = (base_h, base_w)
    closest_cell = find_closest(matrix, start)
    matrix = img
    results = []
    for coord in coords:
        t_h, t_w = get_pixels_by_coords(coord[0], coord[1],56.07461313, 34.43509312)
        pix_h, pix_w = base_h+t_h, base_w+t_w
        end = (pix_h, pix_w)  
        start = (closest_cell) 
        result_matrix = find_three_shortest_paths(matrix, start, end, 2)
        results.append(result_matrix)
        
    sat = img
    img2 = mask
    temp  = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    thickness = 5
    for result_matrix in results:
        for i in range(1,len(result_matrix[0][0])):
            temp = cv2.line(temp, (result_matrix[0][0][i-1][1],result_matrix[0][0][i-1][0]), (result_matrix[0][0][i][1],result_matrix[0][0][i][0]), (255,0,0), thickness)
        temp = cv2.line(temp, (base_w, base_h),(closest_cell[1],closest_cell[0]),(255,0,0),2)
        temp = cv2.rectangle(temp, (base_w-5, base_h-5, 10, 10),(255,0,0),-1)
     
    patch_size=(800,800)

    img_patches = []
    for y in range(0, sat.shape[0], patch_size[1]):
        for x in range(0, sat.shape[1], patch_size[0]):
            img_patch = sat[y:y + patch_size[1], x:x + patch_size[0]]
            img_patches.append(cv2.resize(img_patch, (512, 512)))     

    vis_h_img = []
    h_steps = int(sat.shape[1]/patch_size[1])
    v_steps = int(sat.shape[0]/patch_size[0])
    print(h_steps, v_steps)
    h_1 = 0
    h_2 = h_steps
    for _ in range(v_steps):
        vis_h_img.append(cv2.hconcat([img_patches[i] for i in range(h_1, h_2)]))
        h_1 = h_2
        h_2 += h_steps
    edited_image = cv2.vconcat([vis_h_img[i] for i in range(len(vis_h_img))])
    
    color = [i/255.0 for i in (100, 20, 100)]
    np.place(img2[:, :, :], temp[:, :, :] >= 100, color)
    img2 = cv2.line(img2, (base_w, base_h),(closest_cell[1],closest_cell[0]),(255,0,0),2)
    img2 = cv2.rectangle(img2, (base_w-5, base_h-5, 10, 10),(255,0,0),-1)
    
    #roadmap_on_sat = placeMaskOnImg(placeMaskOnImg(img[0], np.repeat(np.expand_dims(pred_o,-1), 3, 2), (0,255,0)),temp,(0,0,0))
    print(edited_image.shape, img2.shape)
    #color = [i/255.0 for i in (255, 255, 255)]
    #print(img2.shape, temp.shape)
    np.place(edited_image[:, :, :], np.repeat(np.expand_dims(img,-1), 3, 2)[:, :, :] >= 0.5, (100, 100, 0))
    np.place(edited_image[:, :, :], temp[:, :, :] >= 100, (255, 255, 255))

    
    return 'Path on graph',img2 , 'Satellite roadmap and path', edited_image #roadmap_on_sat

In [109]:
my_app = gr.Blocks()

UI functionality

In [110]:
with my_app:
    gr.Markdown("Aerial human segmentation")
    with gr.Tabs():
        with gr.TabItem("Find humans"):
            with gr.Row():
                with gr.Column():
                    drone_img_source = gr.Image(label="Please select source Image")
                    drone_source_image_loader = gr.Button("Find humans")
                    get_satellite_map = gr.Button("Get satellite map")
                with gr.Column():
                    drone_output_label_mask = gr.Label(label="Image Info")
                    drone_mask_output = gr.Image(label="Image Mask")
            with gr.Row():
                with gr.Column():
                    number_label = gr.Label(label="Number of humans")
        with gr.TabItem("Satellite map and roads"):
            with gr.Row():
                with gr.Column():
                    map_unlabelled = gr.Image(label="Please select source Image")
                    road_source_image_loader = gr.Button("Find roads")
                with gr.Column():
                    road_output_label_mask = gr.Label(label="Image Info")
                    road_mask_output = gr.Image(label="Image Mask")
                with gr.Column():
                    road_output_label_img = gr.Label(label="Image Info")
                    road_img_output = gr.Image(label="Mask on Image")
        with gr.TabItem("Find localties"):
            with gr.Row():
                with gr.Column():
                    road_way_img_source = gr.Image(label="Please select source Image")
                    road_way_source_image_loader = gr.Button("Find ways to localties")
                with gr.Column():
                    road_way_output_label_mask = gr.Label(label="")
                    road_way_mask_output = gr.Image(label="")
                with gr.Column():
                    road_way_output_label_img = gr.Label(label="")
                    road_way_img_output = gr.Image(label="")   
    get_satellite_map.click(
        get_satellite_map_from_image,
        [
            drone_img_source,
            number_label
        ],
        [
            road_way_img_source,
            map_unlabelled
        ]
    )         
    drone_source_image_loader.click(
        process_drone_image,
        [
            drone_img_source
        ],
        [
            drone_output_label_mask,
            drone_mask_output,
            number_label
        ]
    )
    road_source_image_loader.click(
        process_road_image,
        [
            map_unlabelled
        ],
        [
            road_output_label_mask,
            road_mask_output,
            road_output_label_img,
            road_img_output
        ]
    )
    road_way_source_image_loader.click(
        process_road_way_image,
        [
            
        ],
        [
            road_way_output_label_mask,
            road_way_mask_output,
            road_way_output_label_img,
            road_way_img_output
        ]
    )
    #map_unlabelled,
    #road_output_label_mask

Launching and closing app

In [113]:
my_app.close()

Closing server running on port: 7860


In [112]:
my_app.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(225, 512)
k_shortest_paths--------------0:00:00.851000
k_shortest_paths--------------0:00:00.730000
k_shortest_paths--------------0:00:00.315000
2 1
(512, 1024, 3) (512, 1024, 3)
1/1 [==============================] - 0s 40ms/step
Found 17 nearby cities and villages.
{'name': 'Shchekoldino', 'address': 'Shchekoldino, Tver Oblast, Russia, 172317', 'lat': 56.0591328, 'lng': 34.4673425}
{'name': 'Country house with farm in Shchekoldino', 'address': "ул. Молодёжная, д. 25, Shchekoldino, Tverskaya oblast', Russia, 172317", 'lat': 56.05581979999999, 'lng': 34.4490034}
{'name': 'Territoriya Dzen', 'address': "Тверская область, Зубцовский район, с/п Щеколдино, ул.Восточная, д.1, Зубцов, Tverskaya oblast', Russia, 172317", 'lat': 56.0688681, 'lng': 34.4525223}
{'name': 'Point of sales Tele2', 'address': "улица Московская, 1A магазин Березка, Shchekoldino, Tverskaya oblast', Russia, 172317", 'lat': 56.055826, 'lng': 34.4654542}
{'name': 'Tomb Of Soviet Soldiers, Fallen In 1941-1943', 'address':